   # Lab 2 - Template

The template is given as starting point to help you get up to speed with loading file and understanding data.  
Use the template and answer each question in the appropriate cell.

Submit:
- eLearning
- Github: create new folder Lab-2. Commit your work under the folder

Naming: First-name_Last_name_ID_Lab_2.ipynb

## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [296]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is data exploration utility

It is up to you if you want to use the function. If you find bugs, please let me know but try to fix them yourself.  
The function most definitely doesn't work with every input

In [297]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [298]:
data = pd.read_csv('SBA_loans_small.csv')
data.head(10)

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"
5,CANTON,OH,44708,APPLE CREEK BK. CO,OH,722211,32,2,2.0,6,2,0,1,N,N,"$75,000.00",$0.00,CHGOFF,"$75,000.00","$67,500.00"
6,Idaho Falls,ID,83404,"READYCAP LENDING, LLC",CA,811118,100,25,2.0,0,25,0,1,N,N,"$603,000.00",$0.00,CHGOFF,"$623,000.00","$467,250.00"
7,TAYLOR,MI,48180,CITIZENS BANK NATL ASSOC,RI,811191,54,3,1.0,0,3,0,1,Y,N,"$300,000.00",$0.00,CHGOFF,"$100,000.00","$50,000.00"
8,MCALLEN,TX,78502,INTER NATIONAL BANK,TX,0,79,6,1.0,0,0,0,0,N,N,"$400,000.00",$0.00,CHGOFF,"$400,000.00","$320,000.00"
9,PHOENIX,AZ,85238,COMMUNITY BANK OF NEVADA,NV,722211,4,7,1.0,0,7,80025,1,Y,N,"$77,820.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"


In [299]:
data.shape

(448583, 20)

In [300]:
desc_df = describe_more(data)


In [301]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24757,object,1,30,"{'LOS ANGELES': 5751, 'HOUSTON': 5102, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65183, 'TX': 35161, 'NY': 28716, 'FL': ..."
2,Zip,28185,int64,1,5,"{90015: 492, 10001: 455, 93401: 385, 90010: 37..."
3,Bank,5156,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43347, 'WELLS F..."
4,BankState,54,object,2,3,"{'CA': 58768, 'NC': 39709, 'IL': 32982, 'OH': ..."
5,NAICS,1290,int64,1,6,"{0: 100776, 722110: 13888, 722211: 9713, 81111..."
6,Term,392,int64,1,3,"{84: 114519, 60: 44888, 240: 43020, 120: 38908..."
7,NoEmp,466,int64,1,4,"{1: 76992, 2: 68936, 3: 45269, 4: 36503, 5: 30..."
8,NewExist,3,float64,3,3,"{1.0: 321541, 2.0: 126457, 0.0: 520}"
9,CreateJob,196,int64,1,4,"{0: 313743, 1: 31439, 2: 29216, 3: 14422, 4: 1..."


## Question 1

How many of the each target label are there? Target label is `MIS_Status` column. 

In [302]:
data.MIS_Status.value_counts()

P I F     369804
CHGOFF     78779
Name: MIS_Status, dtype: int64

#### In the target column MIS_Status, 78779 accounts are classified as  CHGOFF (charged off) and 369804 under PIF (paid in full) loan status

## Question 2

Encode target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.   

Replace values in the original column

In [303]:
pd.get_dummies(data['MIS_Status'])[0:5]

,CHGOFF,P I F
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [304]:
newdata = data.replace({'MIS_Status':{'CHGOFF':1,'P I F':0}})
newdata.head()


,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00"


## Question 3

Think about how to encode `ZIP` column? 

- Should we leave it as is? 
- Should we encode it using one-hot-encoding?
- Should we encode it using target encoder?

#### Question 3a

Copy original Pandas dataframe into new dataframe.  
Encode `ZIP` column using `from sklearn.preprocessing import OneHotEncoder`  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Check new dataframe. Is it good idea to use one-hot-encoder on `ZIP` column? 

In [41]:
data2=data.copy()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448583 entries, 0 to 448582
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   City               448566 non-null  object 
 1   State              448574 non-null  object 
 2   Zip                448583 non-null  int64  
 3   Bank               447822 non-null  object 
 4   BankState          447817 non-null  object 
 5   NAICS              448583 non-null  int64  
 6   Term               448583 non-null  int64  
 7   NoEmp              448583 non-null  int64  
 8   NewExist           448518 non-null  float64
 9   CreateJob          448583 non-null  int64  
 10  RetainedJob        448583 non-null  int64  
 11  FranchiseCode      448583 non-null  int64  
 12  UrbanRural         448583 non-null  int64  
 13  RevLineCr          446257 non-null  object 
 14  LowDoc             447302 non-null  object 
 15  DisbursementGross  448583 non-null  object 
 16  Ba

In [42]:
data2['Zip'] = data2['Zip'].astype(str)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448583 entries, 0 to 448582
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   City               448566 non-null  object 
 1   State              448574 non-null  object 
 2   Zip                448583 non-null  object 
 3   Bank               447822 non-null  object 
 4   BankState          447817 non-null  object 
 5   NAICS              448583 non-null  int64  
 6   Term               448583 non-null  int64  
 7   NoEmp              448583 non-null  int64  
 8   NewExist           448518 non-null  float64
 9   CreateJob          448583 non-null  int64  
 10  RetainedJob        448583 non-null  int64  
 11  FranchiseCode      448583 non-null  int64  
 12  UrbanRural         448583 non-null  int64  
 13  RevLineCr          446257 non-null  object 
 14  LowDoc             447302 non-null  object 
 15  DisbursementGross  448583 non-null  object 
 16  Ba

In [43]:
data_x = data2[['Zip']]
data_x

,Zip
0,80302
1,98101
2,85023
3,77074
4,88001
...,...
448578,92649
448579,3820
448580,79423
448581,61951


In [44]:
from sklearn.preprocessing import OneHotEncoder


In [45]:
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(data_x)
X

<448583x28185 sparse matrix of type '<class 'numpy.float64'>'
	with 448583 stored elements in Compressed Sparse Row format>

In [18]:
# view the resulting dataset
X.toarray()
dfOneHot = pd.DataFrame(X, columns = ["Zip_"+str(int(i)) for i in range(data2.shape[1])]) 
df = pd.concat([data2, dfOneHot], axis=1)
#droping the Zip column 
df= data2.drop(['Zip'], axis=1) 
#printing to verify 
print(df.head())

MemoryError: Unable to allocate 94.2 GiB for an array with shape (448583, 28185) and data type float64

### Comments on encoding Zipcode column:
a) Using One-Hot encoding, we create n-1 dummy variables for n elements in the Zip column. This means addition of new n-1 columns to the dataframe.
So, it leads to increase in the number of dimensions. One-Hot Encoding is not advisable.

b) Using Target Encoding, we assign rank to the various categorical variables (here, unique values in Zip column) based on mean value of the target 
variable against each category. This method doesn't lead to dimensionality issues. But, it is highly dependent on the distribution of data. Whether 
more values with MIS_Status = 1 or MIS_Status = 0 are included.

Overall Target encoding is a better alternative for Zip encoding.

### Question 3b

Encode ZIP column using target (frequency) encoding technique. 
Although, not entirely  correct, replace `ZIP` values with ratios of 0/1 in the `MIS_Status` column.
For example, if for ZIP=11222 there 100 records with label `0` and 50 records with label `1` replace with value 50/100=0.5 

Please see video on feature engineering topic posted in Module-2 for the explanation why frequency encoding is not ideal without modifications.

In [305]:
import numpy as np

In [306]:
v = newdata.groupby('Zip').MIS_Status.value_counts().unstack()
v['ratio']=v[1]/v[0]
v.replace({np.nan : 0})

MIS_Status,0,1,ratio
Zip,,,
0,23.0,118.0,5.130435
1,13.0,0.0,0.000000
2,5.0,2.0,0.400000
3,3.0,0.0,0.000000
4,1.0,0.0,0.000000
...,...,...,...
99925,1.0,0.0,0.000000
99926,1.0,0.0,0.000000
99929,8.0,0.0,0.000000


In [311]:
newdata['Zip'] = v['ratio']
newdata.replace({np.nan:0})

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,5.130435,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00"
1,SEATTLE,WA,0.000000,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,0.400000,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00"
3,HOUSTON,TX,0.000000,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,0.000000,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,0.000000,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,N,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00"
448579,DOVER,NH,0.000000,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,N,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00"
448580,LUBBOCK,TX,0.000000,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,N,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00"
448581,SULLIVAN,IL,0.000000,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,Y,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00"


## Question 4

Encode `NewExist` column using `from sklearn import preprocessing.LabelEncoder`. Replace original column. 

In [47]:
from sklearn.preprocessing import LabelEncoder

In [49]:
# Import label encoder 
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'NewExist'. 
data['NewExist']= label_encoder.fit_transform(data['NewExist']) 
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1,0,0,1,0,N,N,"$492,000.00",$0.00,P I F,"$492,000.00","$344,400.00"
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2,0,1,0,1,N,N,"$10,000.00",$0.00,P I F,"$10,000.00","$5,000.00"
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2,1,1,52007,1,N,N,"$68,000.00",$0.00,P I F,"$68,000.00","$57,800.00"
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1,0,0,1,0,N,Y,"$23,200.00",$0.00,P I F,"$23,200.00","$18,560.00"


## Question 5

Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [70]:
for col in data.columns:
    if data[col].isna().any() == True:
        print(col)

City
State
Bank
BankState
NewExist
RevLineCr
LowDoc


In [72]:
df_clean = data.fillna("Unknown")
df_clean

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111,2,7,1.0,0,0,1,1,Y,N,"$74,104.00",$0.00,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115,2,1,1.0,0,1,1,1,Y,N,"$76,955.00",$0.00,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110,5,6,1.0,0,6,1,1,0,N,"$24,985.00",$0.00,CHGOFF,"$25,000.00","$12,500.00"
3,EDMONDS,WA,98026,BANK OF AMERICA NATL ASSOC,NC,333912,32,1,1.0,0,1,0,1,Y,N,"$7,265.00",$0.00,CHGOFF,"$5,000.00","$2,500.00"
4,PHILADELPHIA,PA,19120,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,58,1,1.0,1,1,1,1,Y,N,"$9,972.00",$0.00,CHGOFF,"$10,000.00","$5,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,TARZANA,CA,91356,CAPITAL ONE NATL ASSOC,VA,541511,84,4,1.0,0,4,0,1,N,N,"$35,000.00",$0.00,P I F,"$35,000.00","$17,500.00"
448579,LOS ANGELES,CA,91352,CALIFORNIA BANK & TRUST,CA,422990,300,6,1.0,0,0,1,1,N,N,"$990,000.00",$0.00,P I F,"$990,000.00","$742,500.00"
448580,DANVILLE,NH,3819,BANK OF AMERICA NATL ASSOC,NC,621111,79,1,1.0,1,1,1,2,Y,N,"$40,011.00",$0.00,P I F,"$22,500.00","$11,250.00"
448581,MISHAWAKA,IN,46545,1ST SOURCE BANK,IN,811111,81,10,1.0,0,10,1,1,N,N,"$71,042.00",$0.00,P I F,"$35,000.00","$17,500.00"


In [76]:
for col in df_clean.columns:
    if df_clean[col].isna().any() == True:
        print(col)
    else:
        print("no missing values")

no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values


## Question 6

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [319]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 7 mins
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_srbuj_qyt2ma
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.570 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [320]:
df_h = h2o.import_file('SBA_loans_small.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [321]:
df_h.describe()

Rows:448583
Cols:20




,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,,,53835.004286832125,,,398521.68325817055,110.85133854827303,11.351830096102624,1.2807847176701934,8.801037935008676,11.131210946469226,2761.0083440522717,0.7576658054362292,,,,0.0013375600230060324,,,
maxs,,,99999.0,,,928120.0,569.0,9999.0,2.0,8800.0,8800.0,92006.0,2.0,,,,600.0,,,
sigma,,,31206.49365795488,,,263210.820607375,78.92372956114484,74.55017951410959,0.4519555984078705,243.3426412853135,243.56176534922847,12778.879367037343,0.6462952056079755,,,,0.8958437440779611,,,
zeros,,,141,,,100776,430,3345,520,313743,219626,104260,161211,,,,448577,,,
missing,17,9,0,761,766,0,0,0,65,0,0,0,0,130956,2040,0,5,0,0,0
0,BOULDER,CO,80302.0,WELLS FARGO BANK NATL ASSOC,SD,327122.0,39.0,8.0,1.0,0.0,8.0,1.0,1.0,Y,N,"$60,639.00",0.0,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101.0,HANMI BANK,CA,722211.0,15.0,2.0,1.0,0.0,2.0,1.0,1.0,,N,"$66,000.00",0.0,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023.0,CAPITAL ONE NATL ASSOC,VA,323119.0,62.0,15.0,1.0,0.0,0.0,1.0,1.0,N,N,"$50,000.00",0.0,CHGOFF,"$50,000.00","$25,000.00"


## Description of Imported Dataframe:
The strings are recognised as'enum' data type and the number of missing values are indicated against each column. A total of 10 columns were recognised as enum type. The columns with numeric values that start with '$' were recognised as 'enum'.Moreover, the categorical columns with integer values '0' and '1' are wrongly categorised as 'int' type and their descriptive statistics were calculated.

For variables recognised as 'int' all the descriptive statistics of minimum, mean, standard deviation, maximum along with the number of zeroes is reported at the beginning of data set.


## Question 7

Use `fillna` method to fill missing values for the following columns:
- City
- State
- Bank
- BankState
- NewExist
- RevLineCr
- LowDoc

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/fillnas.html

In [322]:
'''All Dataframe columns '''
df_h.columns

['City',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'DisbursementGross',
 'BalanceGross',
 'MIS_Status',
 'GrAppv',
 'SBA_Appv']

In [323]:
data3=data.copy()
data3=data3[['City','State','Bank','BankState','NewExist','RevLineCr','LowDoc']]
data3
filled = data3.fillna(method="ffill")
filled

,City,State,Bank,BankState,NewExist,RevLineCr,LowDoc
0,BOULDER,CO,WELLS FARGO BANK NATL ASSOC,SD,1.0,Y,N
1,SEATTLE,WA,HANMI BANK,CA,1.0,0,N
2,PHOENIX,AZ,CAPITAL ONE NATL ASSOC,VA,1.0,N,N
3,HOUSTON,TX,JPMORGAN CHASE BANK NATL ASSOC,IL,1.0,Y,N
4,LAS CRUCES,NM,BANK OF AMERICA NATL ASSOC,NM,2.0,N,N
...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,WELLS FARGO BANK NATL ASSOC,SD,1.0,N,N
448579,DOVER,NH,CITIZENS BANK NATL ASSOC,RI,2.0,N,N
448580,LUBBOCK,TX,PLAINSCAPITAL BANK,TX,2.0,N,N
448581,SULLIVAN,IL,FIRST MID-ILLINOIS BK & TRUST,IL,1.0,N,Y


In [324]:
for col in filled.columns:
    if filled[col].isna().any() == True:
        print(col)
    else:
        print("no missing values")

no missing values
no missing values
no missing values
no missing values
no missing values
no missing values
no missing values


## Question 8

Perform target encoding on the following columns:
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


In [328]:
from h2o.estimators import H2OTargetEncoderEstimator

In [339]:
df_h['MIS_Status'] = df_h['MIS_Status'].asfactor()
response='MIS_Status'

# Split the dataset into train and test
train, test = df_h.split_frame(ratios = [.8], seed = 1234)

# Choose which columns to encode
encoded_columns = ["Zip", "City", "State"]

# For k_fold strategy we need to provide fold column
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5, seed=0)

# Train a TE model
df_te = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",
                                       blending=True,
                                       inflection_point=3,
                                       smoothing=10,
                                       noise=0.15,     # In general, the less data you have the more regularization you need
                                       seed=0)

df_te.train(x=encoded_columns,
                 y=response,
                 training_frame=train)

# New target encoded train and test sets
train_te = df_te.transform(frame=train, as_training=True)
test_te = df_te.transform(frame=test, noise=0)

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%


In [340]:
train_te

Zip,City_te,State_te,City,State,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,none,MIS_Status
80302,0.790353,0.747932,BOULDER,CO,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",0,"$30,000.00","$15,000.00",2,CHGOFF
98101,0.908855,0.875862,SEATTLE,WA,HANMI BANK,CA,722211,15,2,1,0,2,1,1,,N,"$66,000.00",0,"$66,000.00","$33,000.00",2,CHGOFF
85023,0.844811,0.827829,PHOENIX,AZ,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",0,"$50,000.00","$25,000.00",2,CHGOFF
88001,1.02603,0.986737,LAS CRUCES,NM,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",0,"$26,000.00","$23,400.00",2,CHGOFF
44708,0.750247,0.799064,CANTON,OH,APPLE CREEK BK. CO,OH,722211,32,2,2,6,2,0,1,N,N,"$75,000.00",0,"$75,000.00","$67,500.00",0,CHGOFF
83404,0.831473,0.9382,Idaho Falls,ID,"READYCAP LENDING, LLC",CA,811118,100,25,2,0,25,0,1,N,N,"$603,000.00",0,"$623,000.00","$467,250.00",2,CHGOFF
48180,0.758393,0.809233,TAYLOR,MI,CITIZENS BANK NATL ASSOC,RI,811191,54,3,1,0,3,0,1,Y,N,"$300,000.00",0,"$100,000.00","$50,000.00",1,CHGOFF
78502,0.899643,0.89524,MCALLEN,TX,INTER NATIONAL BANK,TX,0,79,6,1,0,0,0,0,N,N,"$400,000.00",0,"$400,000.00","$320,000.00",1,CHGOFF
95610,0.675231,0.728533,CITRUS HEIGHTS,CA,BANCO POPULAR NORTH AMERICA,NY,812320,90,2,2,5,7,1,1,,N,"$407,100.00",0,"$407,100.00","$305,325.00",0,CHGOFF
14621,0.78289,0.724356,ROCHESTER,NY,MANUFACTURERS & TRADERS TR CO,NY,812210,11,10,1,0,10,1,1,Y,N,"$50,000.00",0,"$50,000.00","$25,000.00",4,CHGOFF
